In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["PINECODE_API_KEY"]=os.getenv("PINECODE_API_KEY")
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["HUGGINGFACE_API_KEY"]=os.getenv("HUGGINGFACE_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]=os.getenv("LANGCHAIN_TRACING_V2")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [3]:
llm.invoke("Hello, My name is ratn, how are you?")

AIMessage(content="<think>\nAlright, the user introduced themselves as Ratn and asked how I am. I need to respond politely.\n\nI should thank them for asking and express that I'm just a program without feelings, but here to help.\n\nMaybe add something friendly to keep the conversation going.\n\nI'll keep it simple and welcoming.\n</think>\n\nHello Ratn! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 15, 'total_tokens': 120, 'completion_time': 0.473673483, 'prompt_time': 0.000529122, 'queue_time': 0.057536457, 'total_time': 0.474202605}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run--410c0e68-87d6-4f02-9ed3-db052ba6f367-0', usage_metadata={'input_tokens': 15, 'output_tokens': 105, 'total_tokens': 120})

In [16]:
from langchain_community.tools.tavily_search import TavilySearchResults
search_tool = TavilySearchResults(k=3)

In [17]:
search_tool.invoke("What is top 5 news in india today ?")

[{'title': 'Top 5 News Headlines in India | 25 June 2025 - Dailymotion',
  'url': 'https://www.dailymotion.com/video/x9lvops',
  'content': "Stay updated with today's biggest headlines from across India! In this video, we bring you the top 5 trending and most important news stories for June 25, 2025.",
  'score': 0.69746524},
 {'title': 'India News, Latest Breaking News Headlines from India ... - India Today',
  'url': 'https://www.indiatoday.in/india',
  'content': 'The Supreme Court ordered the release of a Madhya Pradesh law student detained under the NSA, calling the preventive detention "wholly untenable" due to multiple procedural lapses.\n\nsupreme court\n\n#### India News\n\n## PM Modi to embark on 5-nation tour in July, attend BRICS summit in Brazil\n\nPrime Minister Narendra Modi will undertake a multi-nation tour in July 2025, visiting Ghana, Trinidad & Tobago, Argentina, Brazil, and Namibia to strengthen bilateral ties and attend the BRICS Summit. [...] ## Deep regret: Air 

In [10]:
code = """
x=10
y=x+10
print(y)
"""

In [8]:
from langchain_experimental.utilities import PythonREPL
repl=PythonREPL()

In [11]:
repl.run(code)

Python REPL can execute arbitrary code. Use with caution.


'20\n'

In [12]:
repl.invoke(code)

AttributeError: 'PythonREPL' object has no attribute 'invoke'

In [19]:
from typing import Annotated
from langchain.tools import tool
@tool
def python_repl_tool(code: Annotated[str, "The python code to execute to generate your chart."]):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [20]:
python_repl_tool.invoke(code)

'Successfully executed:\n\\`\\`\\`python\n\nx=10\ny=x+10\nprint(y)\n\n\\`\\`\\`\nStdout: 20\n\n\nIf you have completed all tasks, respond with FINAL ANSWER.'

In [29]:
members=["researcher","coder"]
options = members+["FINSIH"]
options

['researcher', 'coder', 'FINSIH']

In [25]:
from typing import Literal, TypedDict
class Router(TypedDict):
    next: Literal['researcher', 'coder', 'FINSIH']

In [24]:
from langgraph.graph import MessagesState,StateGraph,END,START
class State(MessagesState):
    next:str

In [30]:
system_prompt = f""""
You are a supervisor, tasked with managing a conversation between the following workers: {members}. 
Given the following user request, respond with the worker to act next. 
Each worker will perform a task and respond with their results and status. 
When finished, respond with FINISH.
"""

In [32]:
print(system_prompt)

"
You are a supervisor, tasked with managing a conversation between the following workers: ['researcher', 'coder']. 
Given the following user request, respond with the worker to act next. 
Each worker will perform a task and respond with their results and status. 
When finished, respond with FINISH.

